In [1]:
#!pip install requests-html requests pandas

In [2]:
import requests
from requests_html import HTML
import pandas as pd
import time
from slugify import slugify

In [3]:
def clean_data(text, keyname=None):
    if keyname == 'votes':
            return text.replace('\nvotes', '')
    return text

In [4]:
def parse_tag_page(html):
    datas = []
    keys = ['question', 'votes']
    classes = ['.question-hyperlink', '.vote']

    question_summaries = html.find(".question-summary")
    
    link_base_url = 'https://stackoverflow.com/'
    
    for q in question_summaries:
        question_data = {}
        for i,_class in enumerate(classes):
            sub_el = q.find(_class, first=True)
            keyname = keys[i]
            question_data[keyname] = clean_data(sub_el.text, keyname=keyname)

            for link in sub_el.links:
                url = str(sub_el.links).replace("{'/", "").replace("'}", "")
                question_data['answer'] = link_base_url + url
        datas.append(question_data)
    return datas


In [5]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        return []
    html_string = r.text
    html = HTML(html=html_string)
    datas = parse_tag_page(html)
    return datas

In [6]:
def scrape_tag(tag = "html", query_filter="Votes", max_pages=3, pagesize=10):   
    base_url = "https://stackoverflow.com/questions/tagged/"
    datas = []
    for p in range(max_pages):
        page_num = p + 1
        url = f"{base_url}{tag}?tab={query_filter}&page={page_num}&page_size={pagesize}"
        datas += extract_data_from_url(url)
        time.sleep(1)
    return datas

In [7]:
datas = scrape_tag(tag='python')
df = pd.DataFrame(datas)
df.head()

,question,answer,votes
0,"What does the ""yield"" keyword do?",https://stackoverflow.com/questions/231767/wha...,11732
1,"What does if __name__ == ""__main__"": do?",https://stackoverflow.com/questions/419163/wha...,7226
2,Does Python have a ternary conditional operator?,https://stackoverflow.com/questions/394809/doe...,7107
3,What are metaclasses in Python?,https://stackoverflow.com/questions/100003/wha...,6527
4,How do I check whether a file exists without e...,https://stackoverflow.com/questions/82831/how-...,6480


In [8]:
df.to_csv("stack.csv", index=False)